**Hack AI 2024**

Dataset(s):
* https://www.kaggle.com/datasets/ninadaithal/imagesoasis

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.layers as layers
import keras.models
from keras.models import Sequential
import pathlib
from sklearn.model_selection import GroupKFold

In [ ]:
im_width = 496
im_height = 248
batch_size = 32

train_ds = keras.utils.image_dataset_from_directory(
    pathlib.Path('Hack_AI_2024_DATA'),
    validation_split = 0.2,
    subset = 'training',
    seed = 123,
    image_size = (im_height,im_width),
    batch_size = batch_size
)

val_ds = keras.utils.image_dataset_from_directory(
    pathlib.Path('Hack_AI_2024_DATA'),
    validation_split = 0.2,
    subset = 'validation',
    seed = 123,
    image_size = (im_height,im_width),
    batch_size = batch_size  
)

class_names = train_ds.class_names
print(class_names)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(100000).prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)

normalization_layer = layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

num_classes = len(class_names)

model = Sequential([
    layers.Rescaling(1./255, input_shape=(im_height, im_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])



model.compile(optimizer = 'adam',
              loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics=['accuracy']
              )

model.summary()

epochs = 100
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs
)

                

